In [ ]:
import glob
import os
import datetime
import af_import
import rst_import_lct
import rst_import_vcf
import af_import
import run_step1
import run_step2
import polygon_import
import export_shp
import subprocess
import shlex
import glob

In [ ]:
os.environ['PGDATABASE'] = 'finn'
os.environ['PGUSER'] = 'docker'
os.environ['PGPASSWORD'] = 'docker'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

In [ ]:
year = 2016

## Create a PostGIS database

In [ ]:
!createdb finn
!psql -d finn -c 'CREATE EXTENSION postgis;'
!pg_lsclusters

## Import land cover type

Get filenames of MCD12Q1 land cover type data

In [ ]:
# tag to identify dataset
tag_lct = 'modlct_%d' % year
ddir = '../downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006'

search_string = "%(ddir)s/%(year)s.01.01/MCD12Q1.A%(year)s001.h??v??.006.*.hdf" % dict(
        ddir = ddir, year=year)

In [ ]:
# grab hdf file names
fnames = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames) )

In [ ]:
rst_import_lct.main(tag_lct, fnames)

## Import vegetation continuous fields

In [ ]:
tag_vcf = 'modvcf_%d' % year

ddir = '../downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006'

search_string = "%(ddir)s/%(year)s.03.0[56]/MOD44B.A%(year)s065.h??v??.006.*.hdf" % dict(
        ddir = ddir, year=year)

# grab hdf file names
fnames = sorted(glob.glob(search_string))

In [ ]:
fnames[:3]

In [ ]:
rst_import_vcf.main(tag_vcf, fnames)

## Import countries of the world shapefile

In [ ]:
if not os.path.exists('all_countries/All_Countries.shp'):
    subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/All_Countries.zip'))
    subprocess.run(shlex.split('unzip All_Countries.zip -d all_countries'))
    subprocess.run(['rm', 'All_Countries.zip'])

In [ ]:
polygon_import.main('regnum')

## Import active fire data

In [ ]:
if not os.path.exists('firms/global_2016/fire_archive_M6_28864.shp'):
    subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/2016-global-DL_FIRE_M6_28864.zip'))
    subprocess.run(shlex.split('unzip 2016-global-DL_FIRE_M6_28864.zip -d firms/global_2016'))
    subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/2016-global-DL_FIRE_M6_28864.zip'))

In [ ]:
tag_af = 'mod_global_%d_v7m' % year
#tag_af = 'vrs_global_%d_v7m' % year
#tag_af = 'modvrs_global_%d_v7m' % year

dt0 = datetime.date(year, 1, 1)
dt1 = datetime.date(year+1, 1, 1)

# source dir
ddir = 'firms'

# shp file names
arcnames = ['M6_28864', ]
#arcnames = ['V1_23582', ]
#arcnames = ['M6_23581', 'V1_23582', ]
fnames = [os.path.join(ddir, 'global_%d' % year, 'fire_archive_%s.shp' % _) for _ in arcnames]

af_import.main(tag_af, fnames)